In [18]:
from dotenv import dotenv_values
from SPARQLWrapper import SPARQLWrapper, POST, JSON, CSV

In [5]:
env_config = dotenv_values("../.env")

In [6]:
PREFIXES = """
PREFIX  :     <http://www.semanticweb.org/hackathon/ontology/>
PREFIX  owl:  <http://www.w3.org/2002/07/owl#>
PREFIX  rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  xml:  <http://www.w3.org/XML/1998/namespace>
PREFIX  xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX  rdfs: <http://www.w3.org/2000/01/rdf-schema#>
"""

In [7]:
sparql = SPARQLWrapper(
    f"http://{env_config['GRAPHDB_HOST']}:{env_config['GRAPHDB_PORT']}"
    f"/repositories/{env_config['GRAPHDB_REPOSITORY']}"
)
sparql.setMethod(POST)

## Caso de uso 1

Imaginemos que un abogado quiere apelar a una ley definida en `BOE-A-2023-19587`, pero no recuerda si esta sigue vigente o no. Para ello, podría hacer uso de la información almacenada en GraphDB, pues guardamos las relaciones entre entradas.

Podemos primero consultar todas las posibles relaciones que hay ahora mismo.

In [22]:
query = """
SELECT ?rel ?name ?code
WHERE {
    ?rel rdf:type :RelacionAnterior .
    ?rel :nombre ?name .
    ?rel :codigo ?code .
}
"""
sparql.setReturnFormat(JSON)
sparql.setQuery(PREFIXES + query)
results = sparql.query().convert()

In [23]:
for result in results["results"]["bindings"]:
    print(result["rel"]["value"], result["name"]["value"], result["code"]["value"])

http://www.semanticweb.org/hackathon/ontology/deroga DEROGA 210
http://www.semanticweb.org/hackathon/ontology/deConformidadCon DE CONFORMIDAD con 440
http://www.semanticweb.org/hackathon/ontology/cita CITA 330
http://www.semanticweb.org/hackathon/ontology/modifica MODIFICA 270
http://www.semanticweb.org/hackathon/ontology/corrigeErrores CORRIGE errores 203
http://www.semanticweb.org/hackathon/ontology/correcciónDeErrores CORRECCIÓN de errores 201
http://www.semanticweb.org/hackathon/ontology/amplía AMPLÍA 406
http://www.semanticweb.org/hackathon/ontology/correcciónDeErratas CORRECCIÓN de erratas 202
http://www.semanticweb.org/hackathon/ontology/prorroga PRORROGA 401
http://www.semanticweb.org/hackathon/ontology/publica PUBLICA 300
http://www.semanticweb.org/hackathon/ontology/dejaSinEfecto DEJA SIN EFECTO 230
http://www.semanticweb.org/hackathon/ontology/enRelaciónCon EN RELACIÓN con 331
http://www.semanticweb.org/hackathon/ontology/aprueba APRUEBA 420
http://www.semanticweb.org/hackat

Vemos que "SUSPENDE", "ANULA" y "DEJA SIN EFECTO" son las relaciones que nos pueden interesar.

In [28]:
query = """
SELECT ?obj ?rel
WHERE {
    ?obj ?rel :BOE-A-2023-19587 .
    ?rel :codigo ?code .
    FILTER (?code IN (231, 220, 230) )
}
"""
sparql.setReturnFormat(JSON)
sparql.setQuery(PREFIXES + query)
results = sparql.query().convert()

In [30]:
for result in results["results"]["bindings"]:
    print(result["obj"]["value"], result["rel"]["value"])

http://www.semanticweb.org/hackathon/ontology/BOE-A-2023-20078 http://www.semanticweb.org/hackathon/ontology/dejaSinEfecto


Vemos que efectivamente esta entrada fue dejada sin efecto por otra posterior.

## Caso de uso 2

Otro posible caso de uso sería encontrar todas las sentencias relacionadas aplicables en Las Rozas, Madrid. Esto implica obtener todas aquellas vigentes en el país, Madrid o Las Rozas.

In [40]:
query = """
SELECT ?obj
WHERE {
    ?obj :origenLegislativo ?orig .
    FILTER(?orig IN (:España, :ComunidadDeMadrid, :AyuntamientoDeLasRozasDeMadrid))
}
"""
sparql.setReturnFormat(JSON)
sparql.setQuery(PREFIXES + query)
results = sparql.query().convert()

In [41]:
for result in results["results"]["bindings"]:
    print(result["obj"]["value"])

http://www.semanticweb.org/hackathon/ontology/BOE-A-2023-10683
http://www.semanticweb.org/hackathon/ontology/BOE-A-2023-10697
http://www.semanticweb.org/hackathon/ontology/BOE-A-2023-10696
http://www.semanticweb.org/hackathon/ontology/BOE-A-2023-10682
http://www.semanticweb.org/hackathon/ontology/BOE-A-2023-10694
http://www.semanticweb.org/hackathon/ontology/BOE-A-2023-10643
http://www.semanticweb.org/hackathon/ontology/BOE-A-2023-10642
http://www.semanticweb.org/hackathon/ontology/BOE-A-2023-10681
http://www.semanticweb.org/hackathon/ontology/BOE-A-2023-10695
http://www.semanticweb.org/hackathon/ontology/BOE-A-2023-10691
http://www.semanticweb.org/hackathon/ontology/BOE-A-2023-10685
http://www.semanticweb.org/hackathon/ontology/BOE-A-2023-10652
http://www.semanticweb.org/hackathon/ontology/BOE-A-2023-10646
http://www.semanticweb.org/hackathon/ontology/BOE-A-2023-10647
http://www.semanticweb.org/hackathon/ontology/BOE-A-2023-10653
http://www.semanticweb.org/hackathon/ontology/BOE-A-202

Esto se podría hacer realmente con MongoDB directamente también. Un caso de uso más complejo que sí requiera de GraphDB sería buscar sentencias aplicables en distintos sitios de Madrid. Para ello, deberíamos unir nuestra base a *dbpedia* para acceder a aquellas localidades que pertenezcan a Madrid.